In [1]:
import numpy as np
from scipy.stats import multivariate_normal, binom
import pandas as pd
import math
from scipy.special import logsumexp, softmax
from scipy.optimize import minimize, Bounds, LinearConstraint
import itertools
from itertools import combinations,permutations
import matplotlib.pyplot as plt
import random
from scipy.linalg import cholesky
import json
import pickle
import os

class HiddenSemiMarkovDDC:
    def __init__(self, transition, weights,means,covariances, sojourn_time,n_actions=3):

        # P(s' | s, a). an array of 2 * 2 * 3
        self.P = transition

        # P(z | s).
        self.Q_weights = weights
        self.Q_means = means
        self.Q_covariances = covariances

        # binomial distribution
        self.ST1 = np.array([binom.pmf(i, 74, sojourn_time[0]) for i in range(75)])
        self.ST2 = np.array([binom.pmf(i, 14, sojourn_time[1]) for i in range(15)])

        self.S = 2
        self.Z = 2
        self.M = 1
        self.A = n_actions #2,10

        self.log_offset = -700
        self.min_log_acc = -1500

    def params_to_covariance_exponential(self, matrix):
      # Ensure matrix is symmetric
      matrix = (matrix + matrix.T) / 2
      # Exponentiate to ensure positive definiteness
      covariance_matrix = np.exp(matrix)
      return covariance_matrix

    # Convert parameters to covariance matrix using Cholesky decomposition
    def params_to_covariance_cholesky(self, matrix):
      L = matrix
      # Ensure L is a lower triangular matrix
      L = np.tril(L)
      # Construct the covariance matrix
      covariance_matrix = np.dot(L, L.T)  # LL^T is positive definite
      return covariance_matrix

    def GMM_Q(self, z):
      Q = np.zeros((self.S,))
      for state in range(self.S):
        emission_prob = 0
        for mix in range(self.M):
          #add regularization
          cov_mat = self.Q_covariances[state, mix] + np.eye(self.Q_covariances[state, mix].shape[0]) * np.finfo(float).eps
          cov_mat = self.params_to_covariance_cholesky(cov_mat)#cholesky(cov_mat, lower=True)#self.params_to_covariance_exponential(cov_mat)
          emission_prob += self.Q_weights[state, mix] * multivariate_normal.pdf(z,mean=self.Q_means[state, mix],
                                                                          cov=cov_mat,
                                                                          #allow_singular=True
                                                                          )
        Q[state] = emission_prob
        #print('Q', Q)
      #print('Q',Q)
      #print('soft',softmax(Q))
      #Q = Q / Q.sum()
      #Q = softmax(Q)
      return Q

    def sigma(self, x0, x1, a, z):
        Q = self.GMM_Q(z)
        #print(Q,'sigma')
        y = np.array([x0[0],x1[0]])
        part1 = y @ self.P[a] @ Q
        y = np.array([x0[1:].sum(),x1[1:].sum()])
        part2 = y @ Q
        return part1 + part2

    def update_belief(self, x0, x1, a, z):
        denom = max(self.sigma(x0, x1, a, z), np.finfo(float).eps)
        Q = self.GMM_Q(z)
        #print(Q,'belief')
        #tau = 0
        y = np.array([x0[0],x1[0]])
        y1 = Q[0]* self.ST1
        y2 = Q[1]* self.ST2
        state_trans = y @ self.P[a]
        tau_tran_s0 = state_trans[0]*y1
        tau_tran_s1 = state_trans[1]*y2
        #tau = t+1
        bel_trans_s0 = np.append(Q[0]*x0[1:],[0])
        bel_trans_s1 = np.append(Q[1]*x1[1:],[0])


        s0_new = (tau_tran_s0+bel_trans_s0)/denom
        s1_new = (tau_tran_s1+bel_trans_s1)/denom

        assert denom > 0, f'denom = {denom}, ERROR! ' \
            f'x = {x}, a = {a}, z = {z}. P = {self.P}, Q = {Q}, ST = {self.ST}'
        return s0_new, s1_new

In [2]:
class estimate_dynamics:
   def __init__(self, data):
      self.data = data
      self.n_states = 2
      self.n_actions = 3
      self.n_mixtures = 1   
      self.n_features = 2
      self.cov_type = False
      self.vert_sum = None
      self.beliefs = None 
      self.sigmas = None

      self.no_of_params = self.n_states * self.n_actions  # n_states* n_states* n_actions     #states
      self.no_of_params += self.n_states * self.n_mixtures  # weights
      self.no_of_params += self.n_states * self.n_mixtures * self.n_features  # means
      if self.cov_type == True:
        self.no_of_params += self.n_states * self.n_features * self.n_features  # covariances
      else:
        self.no_of_params += self.n_states * self.n_features  # variances
        self.no_of_params += self.n_states

   def extract_parameters(self,parameters):
      num_params_trans_matrix = self.n_states* self.n_actions
      num_params_Q_weights = self.n_states * self.n_mixtures
      num_params_Q_means = self.n_states * self.n_mixtures * self.n_features
      if self.cov_type == True:
        #if Full type covariance
        num_params_Q_covariances = self.n_states * self.n_mixtures* self.n_features* self.n_features
      else:
        #if Diag covariance
        num_params_Q_covariances = self.n_states *self.n_mixtures* self.n_features
        
      start = 0
      stop = num_params_trans_matrix
      params_ = parameters[start:stop]
      #print("params_",params_)
      P = np.zeros((self.n_actions, 2, 2))

      # Fill P dynamically
      for action in range(self.n_actions):
          P[action, 0, 0] = params_[2 * action]  # Probability of staying in state 0
          P[action, 0, 1] = 1 - params_[2 * action]  # Probability of moving to state 1
          P[action, 1, 0] = params_[2 * action + 1]  # Probability of moving to state 0 from 1
          P[action, 1, 1] = 1 - params_[2 * action + 1]  # Probability of staying in state 1
          
      start = num_params_trans_matrix
      stop = num_params_trans_matrix+num_params_Q_weights
      Q_weights = parameters[start:stop].reshape(self.n_states, self.n_mixtures)
      start = stop
      stop = start+num_params_Q_means
      Q_means = parameters[start:stop].reshape(self.n_states, self.n_mixtures, self.n_features)
      start = stop
      stop = start+num_params_Q_covariances
      if self.cov_type == True:
        #if FULL cov
        Q_covariances = parameters[start:stop].reshape(self.n_states, self.n_mixtures, self.n_features, self.n_features)
      else:
        #if Diag cov
        variances = parameters[start:stop].reshape(self.n_states, self.n_mixtures, self.n_features)
        # Initialize the covariance matrix with zeros
        Q_covariances = np.zeros((self.n_states, self.n_mixtures, self.n_features, self.n_features))
        # Fill the diagonal with variances
        for state in range(self.n_states):
          for mixture in range(self.n_mixtures):
            np.fill_diagonal(Q_covariances[state, mixture], variances[state, mixture])
      start = stop
      ST = parameters[start:]

      return P, Q_weights, Q_means, Q_covariances, ST

   def constraint_eq(self,params):
      num_params_trans_matrix = self.n_states * self.n_actions
      num_params_Q_weights = self.n_states * self.n_mixtures
      start = num_params_trans_matrix
      stop = start+num_params_Q_weights
      Q_weights = params[start:stop].reshape(self.n_states, self.n_mixtures)
      # Constraints to ensure each row in the last dimension sums to 1
      return np.concatenate([np.sum(Q_weights, axis=1).flatten() - 1])

   def ll_dynamic(self, parameters, data, init_b0, init_b1):
      P, Q_weights, Q_means, Q_covariances, ST = self.extract_parameters(parameters)
      agent = HiddenSemiMarkovDDC(P, Q_weights,Q_means,Q_covariances, ST,self.n_actions)

      def ll_one(sample_path, init_b0, init_b1):
          # this function calculated log likelihood for one sample path
          ll = 0
          x0, x1 = init_b0, init_b1
          #print(x)
          for cell in sample_path:
              a, z1,z2 = cell
              z = np.array([z1,z2])
              a = int(a)
              #print('a,z',a,z)
              sigma = max(agent.sigma(x0, x1, a, z), np.finfo(float).eps)
              assert sigma > 0, f'probability = {sigma} negativity or zero ERROR!'
              #assert sigma < 1, f'probability = {sigma} ERROR!'
              ll += np.log(sigma)
              x0, x1 = agent.update_belief(x0, x1, a, z)
              #print(x0.shape,x1.shape)
          return ll

      ll = 0  # start calculating total log likelihood
      for i, history in enumerate(data):  # histories w.r.t. i-th initial belief
          ll += ll_one(history, init_b0, init_b1)
          #print(i)

      return -ll  # max: log-likelihood <=> min: -log-likelihood
   
   def get_belief(self,parameters, data, init_b0, init_b1):
      P, Q_weights, Q_means, Q_covariances, ST = self.extract_parameters(parameters)
      agent = HiddenSemiMarkovDDC(P, Q_weights,Q_means,Q_covariances, ST,self.n_actions)

      def ll_one(sample_path, init_b0, init_b1):
          # this function calculated log likelihood for one sample path
          x0, x1 = init_b0, init_b1
          dict_x0 = []
          dict_x1 = []
          dict_x0.append(sum(x0))
          dict_x1.append(sum(x1))
          for cell in sample_path:
              a, z1,z2 = cell
              z = np.array([z1,z2])
              a = int(a)
              sigma = max(agent.sigma(x0, x1, a, z), np.finfo(float).eps)
              assert sigma > 0, f'probability = {sigma} negativity or zero ERROR!'
              #assert sigma < 1, f'probability = {sigma} ERROR!'
              x0, x1 = agent.update_belief(x0, x1, a, z)
              dict_x0.append(sum(x0))
              dict_x1.append(sum(x1))
          return dict_x0,dict_x1

      d_x0, d_x1 = [],[]
      for i, history in enumerate(data):  # histories w.r.t. i-th initial belief
          dict_x0,dict_x1 = ll_one(history, init_b0, init_b1)
          d_x0.append(dict_x0)
          d_x1.append(dict_x1)
      return d_x0, d_x1
   
   def observation_sampler(self,agent,x0,x1,num_samples):
      state = np.random.choice(agent.S, p=[sum(x0),sum(x1)])
      mix = np.random.choice(agent.M, p=agent.Q_weights[state])
      # Sample from the obs distribution
      obs = multivariate_normal.rvs(mean=agent.Q_means[state, mix],
                              cov=agent.Q_covariances[state, mix],
                              size = num_samples)
      return obs
   
   def compute_beliefs_and_sigmas(self,agent, num_samples=100):
      # Create caches
      belief_cache = {}
      sigma_cache = {}

      # Create the states
      vertices = np.eye(90)  # 90, n_bel(or bstates)
      vert_ = []
      vert_sum = []
      
      for i in range(len(vertices)):
          vert = vertices[i]
          v1 = vert[:75]  # :75, n_bstates for first ST
          v2 = vert[75:]  # 75:, n_bstates for second ST
          vert_.append([v1, v2])
          vert_sum.append([v1.sum(), v2.sum()])
      
      vert_sum = np.array(vert_sum)

      # Store sigmas, beliefs
      for x in range(len(vert_)):
          for a in range(agent.A):
              obs_MC = self.observation_sampler(agent,vert_[x][0], vert_[x][1], num_samples)
              for i, z in enumerate(obs_MC):
                  key = (tuple(vert_[x][0]) + tuple(vert_[x][1]), a, i)
                  x_next0, x_next1 = agent.update_belief(vert_[x][0], vert_[x][1], a, z)
                  sigma_cache[key] = agent.sigma(vert_[x][0], vert_[x][1], a, z)
                  belief_cache[key] = np.concatenate((x_next0, x_next1))

      # Get vectorized beliefs
      keys = list(belief_cache.keys())
      num_keys = len(keys)
      array_shape = belief_cache[keys[0]].shape
      
      four_d_array = np.empty((num_keys, *array_shape))
      for i, key in enumerate(keys):
          four_d_array[i] = belief_cache[key]
      
      beliefs = four_d_array.reshape(90, self.n_actions, num_samples, 90)  # n_states, n_acts, n_obs, n_bel
      
      # Get vectorized sigmas
      keys = list(sigma_cache.keys())
      num_keys = len(keys)
      array_shape = sigma_cache[keys[0]].shape
      
      four_d_array = np.empty((num_keys, *array_shape))
      for i, key in enumerate(keys):
          four_d_array[i] = sigma_cache[key]
      
      sigmas_before_norm = four_d_array.reshape(90, self.n_actions, num_samples)  # n_states, n_acts, n_obs
      
      # Normalize sigmas
      sum_along_last_dim = np.sum(sigmas_before_norm, axis=2, keepdims=True)
      sigmas = sigmas_before_norm / sum_along_last_dim
      
      self.vert_sum = vert_sum
      self.beliefs = beliefs
      self.sigmas = sigmas
      return 0
   
   def value_function(self,reward_matrix,beta):
      Q_val = self.vert_sum @ reward_matrix
      V = logsumexp(Q_val, axis=1)
      #count = 0
      while True:
          # Multiply using einsum
          inter = np.einsum('ijkl,lm->ijkm', self.beliefs, Q_val)
          EV = logsumexp(inter, axis=-1)
          # Compute the dot product across the last dimension
          future_ = np.einsum('ijk,ijk->ij', EV, self.sigmas)

          Q_next = self.vert_sum @ reward_matrix + beta * future_
          V_next = logsumexp(Q_next, axis=1)

          condition = abs(V_next - V).sum()
          Q_val = Q_next
          V = V_next
          if condition < 0.01:
            break
          #count+=1
          #print('count',count,'condition',condition)

      return Q_val, V
   
   def reward(self,w, B, C, s, a):
      return w[s] * B[a] + (1 - w[s]) * (-C[a])
   
   def ll_ccp(self,parameters,agent, data, init_b0, init_b1, beta=0.9):
    #r = np.zeros((2, self.n_actions))
    # Fill the remaining elements from the vector
    #r[0,0]=0
    #r[1,0]=0
    #r[:, 1:] = parameters.reshape(2, self.n_actions-1)
    B = {0: 0,1: 4.1, 2: 5.3}
    C = {0: 0,1: 3, 2: 6.2}
    r = np.zeros((2, self.n_actions))
    for s in [0, 1]:
        for a in range(self.n_actions):
            if a == 0:
                r[s, a] = 0  # already set, explicitly
            else:
                r[s, a] = self.reward(parameters, B, C, s, a)
    Q_val, _ = self.value_function(r, beta)
    def ll_one(sample_path, init_b0, init_b1):
        # this function calculated log likelihood for one sample path
        ll = 0
        x0, x1 = init_b0, init_b1
        for cell in sample_path:
            a, z1, z2 = cell
            z = np.array([z1,z2])
            a = int(a)
            belief = np.concatenate((x0, x1))
            Q_x = belief.flatten() @ Q_val
            ccp = max(softmax(Q_x)[a], np.finfo(float).eps)
            assert ccp > 0, f'prob = {ccp}, negativity or zero ERROR!'
            ll += np.log(ccp)
            x0, x1 = agent.update_belief(x0, x1, a, z)
        return ll

    ll = 0
    for i, history in enumerate(data):  # histories w.r.t. i-th initial belief
            ll += ll_one(history, init_b0, init_b1)

    return -ll  # max: log-likelihood <=> min: -log-likelihood

   def pretty_print_arrays(self,P, Q_means, Q_covariances, ST):
      np.set_printoptions(precision=3, suppress=True)
      print("State Transitions:\n", P, "\n")
      print("Means:\n", Q_means, "\n")
      print("Variance:\n", Q_covariances, "\n")
      print("Sojourn Time:\n", ST, "\n")




In [3]:
with open('Data/participant_bounds.json', 'r') as f:
    participant_bounds = json.load(f)

for pid in participant_bounds.keys():
    participant_bounds[pid]['lb'] = np.array(participant_bounds[pid]['lb'])
    participant_bounds[pid]['ub'] = np.array(participant_bounds[pid]['ub'])
    participant_bounds[pid]['init'] = np.array(participant_bounds[pid]['init'])

def fit_model_for_participant(participant_id, data_dir="Data",seed=0):
    print(f"\n=== Processing {participant_id} ===")

    # Load training data
    try:
        with open(f"{data_dir}/{participant_id}_train_data.pkl", "rb") as f:
            train_data = pickle.load(f)
    except Exception as e:
        print(f"Failed to load training data for {participant_id}: {e}")
        return {
            "Participant": participant_id,
            "LL": None,
            "DynamicsParams": None,
            "RewardParams": None
        }

    # Fit dynamics model
    print("Fitting the dynamics model...")
    estimate_d = estimate_dynamics(train_data)
    constraints = {'type': 'eq', 'fun': estimate_d.constraint_eq, 'args': ()}
    lb = participant_bounds[participant_id]['lb']
    ub = participant_bounds[participant_id]['ub']
    bnds = Bounds(lb=lb, ub=ub)
    np.random.seed(seed)
    init_b0 = np.random.rand(75)
    init_b0 /= init_b0.sum()
    init_b1 = np.zeros(15)
    init_p = participant_bounds[participant_id]['init']

    try:
        res = minimize(
            fun=estimate_d.ll_dynamic,
            x0=init_p,
            args=(train_data, init_b0, init_b1),
            bounds=bnds,
            constraints=constraints,
            method='SLSQP',
            options={'disp': 0}
        )
        param_estimates_1 = res.x
        LL = res.fun
    except Exception as e:
        print(f"Skipping {participant_id} (dynamic model failed): {e}")
        return {
            "Participant": participant_id,
            "LL": None,
            "DynamicsParams": None,
            "RewardParams": None
        }

    # Fit reward model
    print("Fitting the reward model...")
    try:
        P, Q_weights, Q_means, Q_covariances, ST = estimate_d.extract_parameters(param_estimates_1)
        estimate_d.pretty_print_arrays(P, Q_means, Q_covariances, ST)
        agent = HiddenSemiMarkovDDC(np.array(P), np.array(Q_weights), np.array(Q_means),
                                    np.array(Q_covariances), np.array(ST))
        estimate_d.compute_beliefs_and_sigmas(agent)
        np.random.seed(seed)
        b = np.random.random(75 + 15)
        b /= b.sum()
        init_b0 = b[:75]
        init_b1 = b[75:]

        x0 = np.random.random(2)
        bnds2 = Bounds([0]*2, [1]*2)

        res2 = minimize(
            fun=estimate_d.ll_ccp,
            x0=x0,
            args=(agent, train_data, init_b0, init_b1),
            bounds=bnds2,
            method='SLSQP',
            options={'disp': 0}
        )
        param_estimates_2 = res2.x
        print("Estimated Rewards:", param_estimates_2)

    except Exception as e:
        print(f"Skipping {participant_id} (reward model failed): {e}")
        param_estimates_2 = None

    return {
        "Participant": participant_id,
        "LL": LL,
        "DynamicsParams": param_estimates_1,
        "RewardParams": param_estimates_2
    }

In [4]:
res1 = fit_model_for_participant('P1')


=== Processing P1 ===
Fitting the dynamics model...


/opt/anaconda3/lib/python3.13/site-packages/scipy/optimize/_slsqp_py.py:435: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  fx = wrapped_fun(x)


Fitting the reward model...
State Transitions:
 [[[0.618 0.382]
  [0.904 0.096]]

 [[0.001 0.999]
  [0.172 0.828]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.706 -0.03 ]]

 [[32.389  0.204]]] 

Variance:
 [[[[0.684 0.   ]
   [0.    0.108]]]


 [[[1.596 0.   ]
   [0.    0.129]]]] 

Sojourn Time:
 [0.847 0.676] 

Estimated Rewards: [0.108 0.375]


In [5]:
res2 = fit_model_for_participant('P2')


=== Processing P2 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.726 0.274]
  [1.    0.   ]]

 [[0.001 0.999]
  [0.84  0.16 ]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.665  0.803]]

 [[32.593  1.202]]] 

Variance:
 [[[[1.139 0.   ]
   [0.    0.14 ]]]


 [[[1.274 0.   ]
   [0.    0.179]]]] 

Sojourn Time:
 [0.767 0.769] 

Estimated Rewards: [0.234 0.455]


In [6]:
res3 = fit_model_for_participant('P3')


=== Processing P3 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.001 0.999]
  [1.    0.   ]]

 [[0.001 0.999]
  [0.028 0.972]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.498  0.213]]

 [[33.791  0.387]]] 

Variance:
 [[[[0.663 0.   ]
   [0.    0.096]]]


 [[[1.504 0.   ]
   [0.    0.155]]]] 

Sojourn Time:
 [0.66  0.724] 

Estimated Rewards: [0.298 0.396]


In [7]:
res4 = fit_model_for_participant('P4')


=== Processing P4 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.244 0.756]
  [0.212 0.788]]

 [[0.001 0.999]
  [0.283 0.717]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.81  -0.223]]

 [[32.     0.022]]] 

Variance:
 [[[[1.398 0.   ]
   [0.    0.086]]]


 [[[2.326 0.   ]
   [0.    0.116]]]] 

Sojourn Time:
 [0.826 0.612] 

Estimated Rewards: [0.104 0.301]


In [8]:
res5 = fit_model_for_participant('P5')


=== Processing P5 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.589 0.411]
  [1.    0.   ]]

 [[0.001 0.999]
  [0.001 0.999]]

 [[0.001 0.999]
  [0.038 0.962]]] 

Means:
 [[[33.796 -0.309]]

 [[31.113 -0.201]]] 

Variance:
 [[[[0.97  0.   ]
   [0.    0.168]]]


 [[[1.432 0.   ]
   [0.    0.16 ]]]] 

Sojourn Time:
 [0.883 0.75 ] 

Estimated Rewards: [0.023 0.178]


In [9]:
res6 = fit_model_for_participant('P6',seed=2)


=== Processing P6 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.001 0.999]
  [1.    0.   ]]

 [[0.316 0.684]
  [0.358 0.642]]

 [[0.001 0.999]
  [0.1   0.9  ]]] 

Means:
 [[[33.775  0.181]]

 [[32.5    0.263]]] 

Variance:
 [[[[0.815 0.   ]
   [0.    0.112]]]


 [[[0.562 0.   ]
   [0.    0.138]]]] 

Sojourn Time:
 [0.712 0.589] 

Estimated Rewards: [0.284 0.348]


In [10]:
res7 = fit_model_for_participant('P7',seed=1)


=== Processing P7 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.757 0.243]
  [0.329 0.671]]

 [[0.001 0.999]
  [0.182 0.818]]

 [[0.1   0.9  ]
  [0.1   0.9  ]]] 

Means:
 [[[33.736 -0.077]]

 [[33.147  0.048]]] 

Variance:
 [[[[2.473 0.   ]
   [0.    0.057]]]


 [[[1.434 0.   ]
   [0.    0.066]]]] 

Sojourn Time:
 [0.709 0.836] 

Estimated Rewards: [0.279 0.39 ]


In [11]:
res8 = fit_model_for_participant('P8')


=== Processing P8 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.109 0.891]
  [1.    0.   ]]

 [[0.001 0.999]
  [0.129 0.871]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.497 -0.072]]

 [[33.85   0.225]]] 

Variance:
 [[[[0.783 0.   ]
   [0.    0.077]]]


 [[[0.908 0.   ]
   [0.    0.153]]]] 

Sojourn Time:
 [0.831 0.516] 

Estimated Rewards: [0.16  0.407]


In [12]:
res9 = fit_model_for_participant('P9',seed=2)


=== Processing P9 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.436 0.564]
  [0.881 0.119]]

 [[0.001 0.999]
  [0.647 0.353]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.73   0.075]]

 [[32.323  0.386]]] 

Variance:
 [[[[0.736 0.   ]
   [0.    0.11 ]]]


 [[[0.786 0.   ]
   [0.    0.15 ]]]] 

Sojourn Time:
 [0.859 0.699] 

Estimated Rewards: [0.146 0.454]


In [13]:
res10 = fit_model_for_participant('P10')


=== Processing P10 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.583 0.417]
  [0.248 0.752]]

 [[0.001 0.999]
  [0.172 0.828]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.655  0.123]]

 [[33.333  0.401]]] 

Variance:
 [[[[0.846 0.   ]
   [0.    0.139]]]


 [[[1.033 0.   ]
   [0.    0.134]]]] 

Sojourn Time:
 [0.842 0.722] 

Estimated Rewards: [0.333 0.379]


In [14]:
res11 = fit_model_for_participant('P11',seed=1)


=== Processing P11 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.605 0.395]
  [0.896 0.104]]

 [[0.02  0.98 ]
  [0.163 0.837]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.715  0.362]]

 [[32.704  0.629]]] 

Variance:
 [[[[2.394 0.   ]
   [0.    0.085]]]


 [[[1.766 0.   ]
   [0.    0.116]]]] 

Sojourn Time:
 [0.826 0.422] 

Estimated Rewards: [0.355 0.48 ]


In [15]:
res12 = fit_model_for_participant('P12')


=== Processing P12 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.497 0.503]
  [0.811 0.189]]

 [[0.001 0.999]
  [0.612 0.388]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.759 -0.008]]

 [[32.4    0.298]]] 

Variance:
 [[[[1.192 0.   ]
   [0.    0.156]]]


 [[[1.285 0.   ]
   [0.    0.174]]]] 

Sojourn Time:
 [0.834 0.779] 

Estimated Rewards: [0.207 0.438]


In [16]:
res13 = fit_model_for_participant('P13')


=== Processing P13 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.025 0.975]
  [1.    0.   ]]

 [[0.266 0.734]
  [0.7   0.3  ]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.762  0.097]]

 [[32.746  0.311]]] 

Variance:
 [[[[0.815 0.   ]
   [0.    0.094]]]


 [[[0.876 0.   ]
   [0.    0.148]]]] 

Sojourn Time:
 [0.859 0.945] 

Estimated Rewards: [0.138 0.352]


In [17]:
res14 = fit_model_for_participant('P14')


=== Processing P14 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.119 0.881]
  [0.643 0.357]]

 [[0.001 0.999]
  [0.001 0.999]]

 [[0.034 0.966]
  [0.001 0.999]]] 

Means:
 [[[33.577 -0.019]]

 [[33.555  0.241]]] 

Variance:
 [[[[0.724 0.   ]
   [0.    0.061]]]


 [[[1.111 0.   ]
   [0.    0.122]]]] 

Sojourn Time:
 [0.768 0.396] 

Estimated Rewards: [0.086 0.376]


In [18]:
res15 = fit_model_for_participant('P15',seed=1)


=== Processing P15 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.256 0.744]
  [0.523 0.477]]

 [[0.001 0.999]
  [0.116 0.884]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.743  0.235]]

 [[32.905  0.358]]] 

Variance:
 [[[[1.54  0.   ]
   [0.    0.095]]]


 [[[2.314 0.   ]
   [0.    0.339]]]] 

Sojourn Time:
 [0.864 0.566] 

Estimated Rewards: [0.05  0.371]


In [19]:
res16 = fit_model_for_participant('P16',seed=3)


=== Processing P16 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.361 0.639]
  [0.9   0.1  ]]

 [[0.3   0.7  ]
  [0.001 0.999]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[33.634  0.225]]

 [[33.398  0.516]]] 

Variance:
 [[[[1.379 0.   ]
   [0.    0.136]]]


 [[[1.755 0.   ]
   [0.    0.244]]]] 

Sojourn Time:
 [0.769 0.946] 

Estimated Rewards: [0.158 0.256]


In [20]:
res17 = fit_model_for_participant('P17',seed=4)


=== Processing P17 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.326 0.674]
  [0.514 0.486]]

 [[0.237 0.763]
  [0.481 0.519]]

 [[0.001 0.999]
  [0.001 0.999]]] 

Means:
 [[[34.298  0.066]]

 [[31.316  0.165]]] 

Variance:
 [[[[1.763 0.   ]
   [0.    0.072]]]


 [[[2.098 0.   ]
   [0.    0.083]]]] 

Sojourn Time:
 [0.601 0.84 ] 

Estimated Rewards: [0.228 0.266]


In [21]:
res18 = fit_model_for_participant('P18')


=== Processing P18 ===
Fitting the dynamics model...
Fitting the reward model...
State Transitions:
 [[[0.48  0.52 ]
  [0.278 0.722]]

 [[0.001 0.999]
  [0.176 0.824]]

 [[0.064 0.936]
  [0.001 0.999]]] 

Means:
 [[[33.542 -0.007]]

 [[33.71   0.2  ]]] 

Variance:
 [[[[0.451 0.   ]
   [0.    0.088]]]


 [[[0.764 0.   ]
   [0.    0.122]]]] 

Sojourn Time:
 [0.883 0.311] 

Estimated Rewards: [0.216 0.367]


In [22]:
def analyze_participants(participant_ids, mean_1_dict, input_dir="Data"):
    all_results = []

    for pid in participant_ids:
        print(f"Processing participant {pid}...")

        # Load test data
        with open(os.path.join(input_dir, f"{pid}_test_data.pkl"), "rb") as f:
            test_data = pickle.load(f)

        #initialize class
        estimate_d = estimate_dynamics(test_data)
        # Use pretrained values for participant

        mean_1 = mean_1_dict[pid]

        # Initialize beliefs
        np.random.seed(4)
        init_b0 = np.random.rand(75)
        init_b0 /= init_b0.sum()

        init_b1 = np.zeros(15)

        # Estimate beliefs
        b0, b1 = estimate_d.get_belief(mean_1, test_data, init_b0, init_b1)
        b0 = [item for sublist in b0 for item in sublist]
        b1 = [item for sublist in b1 for item in sublist]

        # Load true data
        df = pd.read_csv(os.path.join(input_dir, f"{pid}_true_data.csv"))
        df['x0'] = b0
        df['x1'] = b1

        # Extract segments
        segments = []
        current_segment = []
        for _, row in df.iterrows():
            if row['distraction'] == 1:
                current_segment.append([row['x1'], row['glances_greater_than_2_mean'], abs(row['LaneOffset'])])
            else:
                if current_segment:
                    segments.append(current_segment)
                    current_segment = []
        if current_segment:
            segments.append(current_segment)

        segments_array = [np.array(seg) for seg in segments]

        result = []
        for segment in segments_array:
            cond0 = np.any(segment[:, 0] >= 0.5)
            cond1 = np.any(segment[:, 1] > 0)
            cond2 = np.any(segment[:, 2] >= 0.9)
            result.append([cond0, cond1, cond2])
        result_array = np.array(result)

        # Summary statistics
        n_true_distractions = len(result_array)
        n_predicted_distractions = int(np.sum(result_array[:, 0]))
        n_threshold_distractions = int(np.sum(result_array[:, 1]))
        n_true_lane_offsets = int(np.sum(result_array[:, 2]))
        n_lane_model = int(np.sum(result_array[:, 0] & result_array[:, 2]))
        n_lane_thresh = int(np.sum(result_array[:, 1] & result_array[:, 2]))

        # Lead time analysis
        lead_times = []
        for segment in segments_array:
            col0_pred = np.where(segment[:, 0] >= 0.5)[0]
            col1_pred = np.where(segment[:, 1] > 0)[0]
            if len(col0_pred) > 0 and len(col1_pred) > 0:
                idx_0 = col0_pred[0]
                idx_1 = col1_pred[0]
                lead_times.append(idx_1 - idx_0)

        if lead_times:
            lead_time_mean = np.mean(lead_times)
        else:
            lead_time_mean = None

        all_results.append({
            "Participant": pid,
            "TrueDistractionEvents": n_true_distractions,
            "ModelDistractionEvents": n_predicted_distractions,
            "ThresholdDistractionEvents": n_threshold_distractions,
            "TrueLaneOffsetEvents": n_true_lane_offsets,
            "ModelLaneOffsetEvents": n_lane_model,
            "ThresholdLaneOffsetEvents": n_lane_thresh,
            "LeadTimeMean": lead_time_mean,
        })

    # Compile into DataFrame
    results_df = pd.DataFrame(all_results)
    return results_df

with open("Data/pre_trained_values.pkl", "rb") as f:
    pre_trained_dict = pickle.load(f)
participant_ids = [f"P{i}" for i in range(1, 19)]
results_df = analyze_participants(participant_ids, pre_trained_dict)

def print_detection_summary(results_df):
    # Aggregate totals
    total_distraction = results_df["TrueDistractionEvents"].sum()
    model_distraction = results_df["ModelDistractionEvents"].sum()
    rule_distraction = results_df["ThresholdDistractionEvents"].sum()

    total_lane = results_df["TrueLaneOffsetEvents"].sum()
    model_lane = results_df["ModelLaneOffsetEvents"].sum()
    rule_lane = results_df["ThresholdLaneOffsetEvents"].sum()

    # Average lead time (excluding NaNs)
    avg_lead_time = results_df["LeadTimeMean"].dropna().mean()

    # Create summary table
    summary = pd.DataFrame([
        {
            "Event Type": "Distraction Events",
            "Total Events": total_distraction,
            "POSMDP Model (% Detected)": f"{(model_distraction / total_distraction * 100):.1f}%",
            "Baseline Rule (% Detected)": f"{(rule_distraction / total_distraction * 100):.1f}%"
        },
        {
            "Event Type": "Lane Offset Events",
            "Total Events": total_lane,
            "POSMDP Model (% Detected)": f"{(model_lane / total_lane * 100):.1f}%",
            "Baseline Rule (% Detected)": f"{(rule_lane / total_lane * 100):.1f}%"
        }
    ])

    print(summary.to_string(index=False))
    print(f"\nAverage Lead Time (in time steps): {avg_lead_time:.2f} seconds")
    
print_detection_summary(results_df)

Processing participant P1...
Processing participant P2...
Processing participant P3...
Processing participant P4...
Processing participant P5...
Processing participant P6...
Processing participant P7...
Processing participant P8...
Processing participant P9...
Processing participant P10...
Processing participant P11...
Processing participant P12...
Processing participant P13...
Processing participant P14...
Processing participant P15...
Processing participant P16...
Processing participant P17...
Processing participant P18...
        Event Type  Total Events POSMDP Model (% Detected) Baseline Rule (% Detected)
Distraction Events            38                     97.4%                      68.4%
Lane Offset Events            17                    100.0%                      70.6%

Average Lead Time (in time steps): 1.30 seconds
